In [ ]:
from ticker import Ticker, perform_regression
from yfinance_utils import fetch_stock_data, get_sp500_data, parallelize_fetch
from tqdm import tqdm
import polars as pl
import numpy as np


In [ ]:
import pickle
from multiprocessing import Pool
from datetime import date 

def get_ticker_data(ticker):
    return ticker, fetch_stock_data(ticker, start_date='2022-01-01', end_date=date.today().strftime('%Y-%m-%d'))


def parallelize_fetch(tickers, output_file='stock_data.pkl', read_cache=False):
    if read_cache:
        with open(output_file, 'rb') as f:
            return pickle.load(f)
    else:
        with Pool() as pool:
            results = dict(list(tqdm(
                pool.imap(get_ticker_data, tickers),
                total=len(tickers),
                desc="Fetching stock data"
            )))

        
        with open(output_file, 'wb') as f:
            pickle.dump(results, f)
    
    return results

sp500 = get_sp500_data()['Symbol'].to_list() + ['^SPX']
tickers_data = parallelize_fetch(sp500, 'stock_data.pkl', read_cache=False)
from copy import deepcopy
tickers_data_copy = deepcopy(tickers_data)
for k,v in tickers_data_copy.items():
    if len(v) != len(tickers_data['AAPL']):
        print(f"Removing {k} from data")
        del tickers_data[k]
with open('stock_data.pkl', 'wb') as f:
    pickle.dump(tickers_data, f)
    

In [ ]:
tickers_data = {ticker: Ticker(ticker).get_momentum().get_market_beta() for ticker in tqdm(Ticker.DATA_DICT.keys())}


In [ ]:
tickers_data

In [ ]:
def calc_mom (ticker, p1 ,p2):
    date = tickers_data_copy[ticker][p1:p2][-1]['date']
    Y = np.log(pl.DataFrame(tickers_data_copy[ticker])[p1:p2]['close_price'].to_numpy())
    X = np.array(range(1, len(Y)+1))
    model = perform_regression(X, Y)
    score = model.rsquared
    momentum = model.params[1] * score * 252
    return {"date": date, ticker: momentum}

In [ ]:
def get_momentum(ticker):
    return [calc_mom(ticker,p1,p2) for p1, p2 in [(i, i+90) for i in range(0, len(tickers_data_copy[ticker])-90)]]

In [ ]:
with Pool() as pool:
    results = list(tqdm(
        pool.imap(get_momentum, tickers_data_copy.keys()),
        total=len(tickers_data_copy.keys()),
        desc="Parallelizing momentum calculation"
    ))

In [ ]:
from functools import reduce
df = reduce(lambda x, y: x.join(y, on='date', how='left'), [pl.DataFrame(result) for result in results if len(result)])
rogue_cols = df.unpivot().filter(pl.col('value').is_null())['variable'].unique().to_list()
df = df.drop(rogue_cols)

In [ ]:
def create_momentum_factor (arr, filter_func=None):
    if filter_func:
        arr = filter_func(arr)
    arr = np.sort(arr)
    indices = [int(len(arr) * i/10) for i in range(1, 11)]    
    sum_by_decile = np.array([np.sum(arr[indices[i]:indices[i+1]]) for i in range(0, len(indices)-1)])
    return sum_by_decile[-3:].mean() - sum_by_decile[:3].mean()

In [ ]:
momentum = np.array([create_momentum_factor(np.array(x[1:])) for x in df.iter_rows()])
momentum = (momentum / momentum[0]) * 100

In [ ]:
import matplotlib.pyplot as plt
momentum = pl.DataFrame({'date': df['date'], 'momentum': momentum})
plt.plot(momentum['date'], momentum['momentum'])

In [27]:
df[-1].drop('date').unpivot(variable_name='ticker', value_name='momentum').filter(pl.col('momentum')>0).sort('momentum', descending=True)[:10]

ticker,momentum
str,f64
"""VRSN""",0.877446
"""GILD""",0.648542
"""PM""",0.568565
"""GL""",0.563443
"""T""",0.53708
"""ABBV""",0.535829
"""EXC""",0.520412
"""CVS""",0.499042
"""FOXA""",0.467351


In [ ]:
sp500 = get_sp500_data()['Symbol'].to_list()
tickers_data = {ticker: Ticker(ticker).get_momentum().get_market_beta() for ticker in tqdm(Ticker.DATA_DICT.keys())}

In [ ]:
import pickle
with open('tickers.pkl', 'wb') as f:
    pickle.dump(tickers_data, f)

In [ ]:
momentum = pl.DataFrame({
    "ticker": list(tickers_data.keys()),
    "momentum": [tickers_data[ticker].momentum for ticker in tickers_data.keys()],
    "alpha": [tickers_data[ticker].alpha['coef'] for ticker in tickers_data.keys()]
}).sort('alpha', descending=True)[:7]['ticker'].to_list()

In [ ]:

sp500 = sp500[sp500.index.is_in(momentum)]

In [ ]:
def calculate_moving_average(data, window=100):
    return list(map(
        lambda i: sum(data[i:i+window]) / window, 
        range(len(data) - window + 1)
    ))

In [ ]:
# plt.plot(tickers_data['^SPX'].price)
calculate_moving_average(tickers_data['^SPX'].price)

In [ ]:
import matplotlib.pyplot as plt

for ticker in momentum:
    prices = tickers_data[ticker].price 
    rescaled_prices = prices / prices[0]
    plt.plot(rescaled_prices, label=ticker)

plt.legend()


In [ ]:
w.calculate_momentum(strategy='ols')